# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

# %matplotlib inline
# %load_ext pycodestyle_magic

/home/steady/anaconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/home/steady/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.gain = self.__gain_gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.gain = self.__gain_entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.gain = self.__gain_misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        pk_l = l_c / l_s
        pk_r = r_c / r_s
        return 1 - np.sum(pk_l ** 2 + pk_r ** 2, axis=1) # Ваш код в 1 строчку
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        pk_l = l_c / l_s
        pk_r = r_c / r_s
        return -np.sum(pk_l * np.log(pk_l) + pk_r * np.log(pk_r),
                       axis=1) # Ваш код в 1 строчку
        
    def __misclass(self, l_c, l_s, r_c, r_s):
        pk_l = l_c / l_s
        pk_r = r_c / r_s
        return 1 - (np.max(pk_l, axis=1) + np.max(pk_r, axis=1)) # Ваш код в 1 строчку
    
    def __gain_gini(self, a):
        t = np.bincount(a).astype(np.float) / a.size
        return 1 - (t ** 2).sum()
    
    def __gain_entropy(self, a):
        t = np.bincount(a).astype(np.float) / a.size
        return -np.nansum(t * np.log2(t))
    
    def __gain_misclass(self, a):
        t = np.bincount(a) / a.size
        return 1 - np.max(t)
        
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.int(np.sqrt(n_feature))]) # Ваш код в 1 строчку
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.int(np.log2(n_feature))]) # Ваш код в 1 строчку

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature) # Ваш код в 1 строчку
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        '''
        Сортируем x по признакам, соответсвенно перетсавляя классы в y;
        Смотри число элементое, которые обрежем у сампла
        '''
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода?
        '''
        Обрезаем сампл;
        Ищем индексы, на которых меняется класс
        Если таких индексов нет, то возвращаем порог inf, None
        '''
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода?
        '''
        One-hot-encode'им матрицу с полученными индексами
        1 там где сменяется класс;
        В class_increments записываем сколько элементов между сменой классов 
        '''
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        '''
        Считаем количество элементов классов в каждом возможном разбиении
        И какой общий размер классов
        '''
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        '''
        Считаем неорпеделенность для каждого разбиения
        Выбираем наименьшее
        '''
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        '''
        Возвращаем меру неопределенности и порог
        '''
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        
        nums_class = np.bincount(y)
        max_num_class = np.argmax(nums_class)
        
        if y.shape[0] < self.min_samples_split or \
            self.max_depth is not None and depth >= self.max_depth or \
            self.sufficient_share <= np.float(nums_class.max()) / y.size:
                
                self.tree[node_id] = self.LEAF_TYPE, max_num_class, nums_class / y.size    
                return None
            
                
        sel_features = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, i], y) for i in sel_features])

        i_best_threshold = thresholds[:, 0].argmin()
        best_threshold = thresholds[i_best_threshold, 1]

        if best_threshold is None:
            self.tree[node_id] = self.LEAF_TYPE, max_num_class, nums_class / y.size
            return None
        
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, sel_features[i_best_threshold],
                                                best_threshold)
        
        if x_l.size == 0 or x_r.size == 0:
            self.tree[node_id] = self.LEAF_TYPE, max_num_class, nums_class.astype(np.float) / y.size
            return None
        
        self.feature_importances_[i_best_threshold] += \
            self.gain(y) * y.size - self.gain(y_l) * y_l.size - \
            self.gain(y_r) * y_r.size
        
        self.tree[node_id] = self.NON_LEAF_TYPE, sel_features[i_best_threshold], best_threshold
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, pred_f)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, pred_f)
        
    
    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= self.feature_importances_.sum()

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 4.38 ms, sys: 296 µs, total: 4.68 ms
Wall time: 3.22 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 102 ms, sys: 5.53 ms, total: 108 ms
Wall time: 107 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8917748917748917

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8968253968253969

## Подготовка данных Speed Dating Data 

In [9]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')

df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
             'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
             'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o',
             'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o',
             'prob_o', 'met_o'], axis=1)

df.drop_duplicates('iid').age.isnull().sum()
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums',
              'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
              'theater', 'movies', 'concerts', 'music', 'shopping',
              'yoga'], axis=1)

df = df.drop(['expnum'], axis=1)
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
        'amb1_1', 'shar1_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
          'shar1_1']].sum(axis=1)

df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
           'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                     'fun1_1', 'amb1_1', 'shar1_1']].T /
                          df.loc[:, 'temp_totalsum'].T).T * 100

feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
        'amb2_1', 'shar2_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                        'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)

df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
           'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                     'fun2_1', 'amb2_1', 'shar2_1']].T /
                          df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
            .drop(['gender'], axis=1).dropna()

df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
            .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1).dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
features_names = df_pair.columns[1:]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12)

In [11]:
my_clf2 = MyDecisionTreeClassifier(min_samples_split=2)
clf2 = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [12]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
%time clf2.fit(X_train, y_train)

CPU times: user 173 ms, sys: 6.27 ms, total: 179 ms
Wall time: 185 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [13]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
%time my_clf2.fit(X_train, y_train)

CPU times: user 859 ms, sys: 3.56 ms, total: 862 ms
Wall time: 876 ms


## Проверка качества работы на Speed Dating Data

In [14]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=clf2.predict(X_test), y_true=y_test, average='macro')

0.5418973086466883

In [15]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=my_clf2.predict(X_test), y_true=y_test, average='macro')

0.4754786257539994

## Задание 3

In [16]:
pd.Series(
    data=clf2.feature_importances_, index=df_pair.columns[1:]
).sort_values(ascending=False).head(10)

int_corr      0.078024
attr1_1_f     0.029481
fun1_1_f      0.027810
sinc1_1       0.027787
shar2_1       0.027756
attr3_1       0.025381
fun2_1_f      0.025106
amb1_1_f      0.024079
career_c_f    0.023065
amb3_1_f      0.023018
dtype: float64

In [17]:
pd.Series(
    data=my_clf2.feature_importances_, index=df_pair.columns[1:]
).sort_values(ascending=False).head(10)

race          0.136118
int_corr      0.135353
field_cd_f    0.129446
age_f         0.109398
mn_sat_f      0.108159
date          0.085310
income_f      0.069901
go_out        0.063143
shar1_1       0.062932
imprace       0.025913
dtype: float64

## Задание 4

In [18]:
r_clf = RandomForestClassifier()
params = {'max_depth': range(1, 15),
          'min_samples_leaf': range(5, 15),
          'n_estimators': range(10, 100, 10),
          'criterion': ['gini', 'entropy']}

random_search = RandomizedSearchCV(r_clf, param_distributions=params)
random_search.fit(X, y)
print(random_search.best_params_)

{'n_estimators': 80, 'min_samples_leaf': 13, 'max_depth': 7, 'criterion': 'entropy'}
